# Generate only the `processInformation` part of the Schema and JSON Instance

In [31]:
# Produce Python module from SharedDefinitions using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

sd_yaml_schema = "../linkml/data/yaml/linkml_shared_definitions.yaml"
sd_python_file = "../linkml/data/py/linkml_shared_definitions.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=sd_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()


with open(sd_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {sd_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_shared_definitions.py.


In [32]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator


pi_yaml_schema = "../linkml/data/yaml/linkml_processInformation_schema.yaml"
pi_python_file = "../linkml/data/py/linkml_processInformation_schema.py"


# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=pi_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()

with open(pi_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {pi_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_processInformation_schema.py.


In [33]:
# Instance Generation and Loading processInformation EPD YAML

import json
import yaml
from collections import OrderedDict
from data.py.linkml_processInformation_schema import ProcessInformation
from linkml.validator import Validator
from linkml_runtime.loaders.yaml_loader import YAMLLoader


def odict_to_dict(obj):
    """
    Recursively convert an OrderedDict (or list/dict containing OrderedDicts) to a standard dict.

    Parameters:
        obj: An OrderedDict, list, or dict potentially containing OrderedDicts.

    Returns:
        A new structure where all OrderedDict instances are replaced with standard dicts.
    """
    if isinstance(obj, OrderedDict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [odict_to_dict(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    else:
        return obj


def build_minimal_epd_structure(data):
    """
    Build a minimal nested dictionary according to our minimal schema.
    This function assumes that the JSON instance already contains a "processInformation" key.

    Parameters:
        data (dict): A dictionary loaded from a JSON file.

    Returns:
        dict: A dictionary structured with a top-level 'processInformation' key.
    """
    instance_dict = {"processInformation": data.get("processInformation", {})}
    return instance_dict


def process_instance(input_path, schema_path, output_yaml_path):
    """
    Process an instance by loading a minimal JSON instance, converting it, dumping it to YAML,
    validating it against a schema, and loading it with YAMLLoader.

    Steps:
      1. Load a minimal JSON instance containing only the processInformation section.
      2. Convert OrderedDicts to dicts.
      3. Wrap the data in a top-level ProcessInformation structure.
      4. Dump the resulting instance to a YAML file.
      5. Validate the instance using a schema validator.
      6. Load the YAML instance with YAMLLoader.

    Parameters:
        input_path (str): Path to the input JSON file.
        schema_path (str): Path to the YAML schema file.
        output_yaml_path (str): Path where the output YAML file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during validation or instance loading.
    """
    # Load the minimal JSON instance (containing only processInformation)
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    data = odict_to_dict(data)

    # Build the nested instance structure.
    instance_data = build_minimal_epd_structure(data)

    # Dump to YAML
    with open(output_yaml_path, "w", encoding="utf-8") as out:
        yaml.dump(instance_data, out, sort_keys=False, allow_unicode=True)
    print(f"processInformation EPD YAML instance written to: {output_yaml_path}")

    # Validate using the Validator
    try:
        validator = Validator(schema_path, strict=False)
        report = validator.validate(instance_data, "ProcessInformation")
        if report.results:
            print("Validation errors:")
            for result in report.results:
                print(result.message)
        else:
            print("The ProcessInformation instance is valid!")
    except Exception as e:
        print("Exception during validation:")
        print(e)
        raise

    # Load the YAML instance using YAMLLoader
    try:
        loader = YAMLLoader()
        # If the instance is wrapped in a "processInformation" key, unwrap it.
        if "processInformation" in instance_data:
            instance_data = instance_data["processInformation"]
        loader.load(instance_data, target_class=ProcessInformation)
        print("Instance loaded successfully!")
    except Exception as e:
        print("Exception during instance loading via YAMLLoader:")
        print(e)
        raise


# Usage
input_file = "../linkml/data/json/instance_processInformation.json"
schema_file = "../linkml/data/yaml/linkml_processInformation_schema.yaml"
output_yaml = "../linkml/data/yaml/linkml_processInformation_instance.yaml"

process_instance(input_file, schema_file, output_yaml)

processInformation EPD YAML instance written to: ../linkml/data/yaml/linkml_processInformation_instance.yaml
The ProcessInformation instance is valid!
Instance loaded successfully!


In [34]:
# Loading and RDF (JSON-LD, Turtle) Generation processInformation EPD YAML Instance

import yaml
from linkml_runtime.dumpers import RDFLibDumper
from linkml_runtime.loaders import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView
from data.py.linkml_processInformation_schema import ProcessInformation


def generate_rdf(
    schema_path: str,
    instance_path: str,
    output_jsonld_path: str,
    output_turtle_path: str,
) -> None:
    """
    Load a validated LinkML YAML instance, create a SchemaView, and dump the instance to RDF in JSON-LD and Turtle formats.

    Steps:
        1. Load the instance from a YAML file.
        2. Create a SchemaView from the schema file.
        3. Generate RDF in JSON-LD and Turtle formats.
        4. Write the RDF to files.

    Parameters:
        schema_path (str): Path to the LinkML YAML schema file.
        instance_path (str): Path to the validated LinkML YAML instance file.
        output_jsonld_path (str): Path where the output JSON-LD file will be written.
        output_turtle_path (str): Path where the output Turtle file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during instance loading, SchemaView creation, or RDF generation.
    """
    # Load instance from file
    try:
        with open(instance_path, "r", encoding="utf-8") as f:
            loaded_data = yaml.safe_load(f)  # This is now a Python dict
        # Extract "processInformation"
        if "processInformation" in loaded_data:
            loaded_data = loaded_data["processInformation"]
        # Feed loaded_data dictionary into the loader
        instance_obj = YAMLLoader().load(loaded_data, target_class=ProcessInformation)
        print(f"Instance loaded from: {instance_path}")
    except Exception as e:
        print("Exception during instance loading:")
        print(e)
        raise

    # Create SchemaView
    try:
        sv = SchemaView(schema_path)
        print("SchemaView created successfully.")
    except Exception as e:
        print("Exception during SchemaView creation:")
        print(e)
        raise

    # Generate and write RDF (JSON-LD, Turtle)
    try:
        dumper = RDFLibDumper()
        # JSON‑LD format
        # rdf_jsonld = dumper.dumps(instance_obj, schemaview=sv, fmt="json-ld")
        # with open(output_jsonld_path, "w", encoding="utf-8") as rdf_file:
        #     rdf_file.write(rdf_jsonld)
        # print(f"RDF (JSON‑LD) successfully written to: {output_jsonld_path}")
        # Turtle format
        rdf_turtle = dumper.dumps(instance_obj, schemaview=sv, fmt="turtle")
        with open(output_turtle_path, "w", encoding="utf-8") as ttl_file:
            ttl_file.write(rdf_turtle)
        print(f"RDF (Turtle) successfully written to: {output_turtle_path}")
    except Exception as e:
        print("Exception during RDF generation:")
        print(e)
        raise


# Example usage:
schema_path = "../linkml/data/yaml/linkml_processInformation_schema.yaml"
instance_path = "../linkml/data/yaml/linkml_processInformation_instance.yaml"
output_jsonld_path = "../linkml/data/rdf/linkml_processInformation_instance.jsonld"
output_turtle_path = "../linkml/data/rdf/linkml_processInformation_instance.ttl"

generate_rdf(schema_path, instance_path, output_jsonld_path, output_turtle_path)

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Instance loaded from: ../linkml/data/yaml/linkml_processInformation_instance.yaml
SchemaView created successfully.
RDF (Turtle) successfully written to: ../linkml/data/rdf/linkml_processInformation_instance.ttl


# Generate only the `modellingAndValidation` part of the Schema and JSON Instance


In [15]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

sd_yaml_schema = "../linkml/data/yaml/linkml_shared_definitions.yaml"
sd_python_file = "../linkml/data/py/linkml_shared_definitions.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=sd_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()


with open(sd_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {sd_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_shared_definitions.py.


In [17]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

mav_yaml_schema = "../linkml/data/yaml/linkml_modellingAndValidation_schema.yaml"
mav_python_file = "../linkml/data/py/linkml_modellingAndValidation_schema.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=mav_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()

with open(mav_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {mav_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_modellingAndValidation_schema.py.


In [19]:
# Instance Generation and Loading modellingAndValidation EPD YAML

import json
import yaml
from collections import OrderedDict
from data.py.linkml_modellingAndValidation_schema import ModellingAndValidation
from linkml.validator import Validator
from linkml_runtime.loaders.yaml_loader import YAMLLoader


def odict_to_dict(obj):
    """
    Recursively convert an OrderedDict (or list/dict containing OrderedDicts) to a standard dict.

    Parameters:
        obj: An OrderedDict, list, or dict potentially containing OrderedDicts.

    Returns:
        A new structure where all OrderedDict instances are replaced with standard dicts.
    """
    if isinstance(obj, OrderedDict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [odict_to_dict(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    else:
        return obj


def build_minimal_epd_structure(data):
    """
    Build a minimal nested dictionary according to our minimal schema.
    This function assumes that the JSON instance already contains a "modellingAndValidation" key.

    Parameters:
        data (dict): A dictionary loaded from a JSON file.

    Returns:
        dict: A dictionary structured with a top-level 'modellingAndValidation' key.
    """
    instance_dict = {"modellingAndValidation": data.get("modellingAndValidation", {})}
    return instance_dict


def process_instance(input_path, schema_path, output_yaml_path):
    """
    Process an instance by loading a minimal JSON instance, converting it, dumping it to YAML,
    validating it against a schema, and loading it with YAMLLoader.

    Steps:
      1. Load a minimal JSON instance containing only the modellingAndValidation section.
      2. Convert OrderedDicts to dicts.
      3. Wrap the data in a top-level ModellingAndValidation structure.
      4. Dump the resulting instance to a YAML file.
      5. Validate the instance using a schema validator.
      6. Load the YAML instance with YAMLLoader.

    Parameters:
        input_path (str): Path to the input JSON file.
        schema_path (str): Path to the YAML schema file.
        output_yaml_path (str): Path where the output YAML file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during validation or instance loading.
    """
    # Load the minimal JSON instance (containing only modellingAndValidation)
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    data = odict_to_dict(data)

    # Build the nested instance structure.
    instance_data = build_minimal_epd_structure(data)

    # Dump to YAML
    with open(output_yaml_path, "w", encoding="utf-8") as out:
        yaml.dump(instance_data, out, sort_keys=False, allow_unicode=True)
    print(f"modellingAndValidation EPD YAML instance written to: {output_yaml_path}")

    # Validate using the Validator
    try:
        validator = Validator(schema_path, strict=False)
        report = validator.validate(instance_data, "ModellingAndValidation")
        if report.results:
            print("Validation errors:")
            for result in report.results:
                print(result.message)
        else:
            print("The ModellingAndValidation instance is valid!")
    except Exception as e:
        print("Exception during validation:")
        print(e)
        raise

    # Load the YAML instance using YAMLLoader
    try:
        loader = YAMLLoader()
        # If the instance is wrapped in a "modellingAndValidation" key, unwrap it.
        if "modellingAndValidation" in instance_data:
            instance_data = instance_data["modellingAndValidation"]
        loader.load(instance_data, target_class=ModellingAndValidation)
        print("Instance loaded successfully!")
    except Exception as e:
        print("Exception during instance loading via YAMLLoader:")
        print(e)
        raise


# Usage
input_file = "../linkml/data/json/instance_modellingAndValidation.json"
schema_file = "../linkml/data/yaml/linkml_modellingAndValidation_schema.yaml"
output_yaml = "../linkml/data/yaml/linkml_modellingAndValidation_instance.yaml"

process_instance(input_file, schema_file, output_yaml)

modellingAndValidation EPD YAML instance written to: ../linkml/data/yaml/linkml_modellingAndValidation_instance.yaml
The ModellingAndValidation instance is valid!
Instance loaded successfully!


In [20]:
# Loading and RDF (JSON-LD, Turtle) Generation modellingAndValidation EPD YAML Instance

import yaml
from linkml_runtime.dumpers import RDFLibDumper
from linkml_runtime.loaders import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView
from data.py.linkml_modellingAndValidation_schema import ModellingAndValidation


def generate_rdf(
    schema_path: str,
    instance_path: str,
    output_jsonld_path: str,
    output_turtle_path: str,
) -> None:
    """
    Load a validated LinkML YAML instance, create a SchemaView, and dump the instance to RDF in JSON-LD and Turtle formats.

    Steps:
        1. Load the instance from a YAML file.
        2. Create a SchemaView from the schema file.
        3. Generate RDF in JSON-LD and Turtle formats.
        4. Write the RDF to files.

    Parameters:
        schema_path (str): Path to the LinkML YAML schema file.
        instance_path (str): Path to the validated LinkML YAML instance file.
        output_jsonld_path (str): Path where the output JSON-LD file will be written.
        output_turtle_path (str): Path where the output Turtle file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during instance loading, SchemaView creation, or RDF generation.
    """
    # Load instance from file
    try:
        with open(instance_path, "r", encoding="utf-8") as f:
            loaded_data = yaml.safe_load(f)  # This is now a Python dict
        # Extract "modellingAndValidation"
        if "modellingAndValidation" in loaded_data:
            loaded_data = loaded_data["modellingAndValidation"]
        # Feed loaded_data dictionary into the loader
        instance_obj = YAMLLoader().load(
            loaded_data, target_class=ModellingAndValidation
        )
        print(f"Instance loaded from: {instance_path}")
    except Exception as e:
        print("Exception during instance loading:")
        print(e)
        raise

    # Create SchemaView
    try:
        sv = SchemaView(schema_path)
        print("SchemaView created successfully.")
    except Exception as e:
        print("Exception during SchemaView creation:")
        print(e)
        raise

    # Generate and write RDF (JSON-LD, Turtle)
    try:
        dumper = RDFLibDumper()
        # # JSON‑LD format
        # rdf_jsonld = dumper.dumps(instance_obj, schemaview=sv, fmt="json-ld")
        # with open(output_jsonld_path, "w", encoding="utf-8") as rdf_file:
        #     rdf_file.write(rdf_jsonld)
        # print(f"RDF (JSON‑LD) successfully written to: {output_jsonld_path}")
        # Turtle format
        rdf_turtle = dumper.dumps(instance_obj, schemaview=sv, fmt="turtle")
        with open(output_turtle_path, "w", encoding="utf-8") as ttl_file:
            ttl_file.write(rdf_turtle)
        print(f"RDF (Turtle) successfully written to: {output_turtle_path}")
    except Exception as e:
        print("Exception during RDF generation:")
        print(e)
        raise


# Example usage:
schema_path = "../linkml/data/yaml/linkml_modellingAndValidation_schema.yaml"
instance_path = "../linkml/data/yaml/linkml_modellingAndValidation_instance.yaml"
output_jsonld_path = "../linkml/data/rdf/linkml_modellingAndValidation_instance.jsonld"
output_turtle_path = "../linkml/data/rdf/linkml_modellingAndValidation_instance.ttl"

generate_rdf(schema_path, instance_path, output_jsonld_path, output_turtle_path)

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Instance loaded from: ../linkml/data/yaml/linkml_modellingAndValidation_instance.yaml
SchemaView created successfully.
RDF (Turtle) successfully written to: ../linkml/data/rdf/linkml_modellingAndValidation_instance.ttl


# Generate only the `administrativeInformation` part of the Schema and JSON Instance


In [26]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

sd_yaml_schema = "../linkml/data/yaml/linkml_shared_definitions.yaml"
sd_python_file = "../linkml/data/py/linkml_shared_definitions.py"

generator = PythonGenerator(schema=sd_yaml_schema)
python_code = generator.serialize()

with open(sd_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {sd_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_shared_definitions.py.


In [27]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

ai_yaml_schema = "../linkml/data/yaml/linkml_administrativeInformation_schema.yaml"
ai_python_file = "../linkml/data/py/linkml_administrativeInformation_schema.py"

generator = PythonGenerator(schema=ai_yaml_schema)
python_code = generator.serialize()

with open(ai_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {ai_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_administrativeInformation_schema.py.


In [29]:
# Instance Generation and Loading administrativeInformation EPD YAML

import json
import yaml
from collections import OrderedDict
from data.py.linkml_administrativeInformation_schema import AdministrativeInformation
from linkml.validator import Validator
from linkml_runtime.loaders.yaml_loader import YAMLLoader


def odict_to_dict(obj):
    """
    Recursively convert an OrderedDict (or list/dict containing OrderedDicts) to a standard dict.

    Parameters:
        obj: An OrderedDict, list, or dict potentially containing OrderedDicts.

    Returns:
        A new structure where all OrderedDict instances are replaced with standard dicts.
    """
    if isinstance(obj, OrderedDict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [odict_to_dict(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    else:
        return obj


def build_minimal_epd_structure(data):
    """
    Build a minimal nested dictionary according to our minimal schema.
    This function assumes that the JSON instance already contains a "administrativeInformation" key.

    Parameters:
        data (dict): A dictionary loaded from a JSON file.

    Returns:
        dict: A dictionary structured with a top-level 'administrativeInformation' key.
    """
    instance_dict = {
        "administrativeInformation": data.get("administrativeInformation", {})
    }
    return instance_dict


def process_instance(input_path, schema_path, output_yaml_path):
    """
    Process an instance by loading a minimal JSON instance, converting it, dumping it to YAML,
    validating it against a schema, and loading it with YAMLLoader.

    Steps:
      1. Load a minimal JSON instance containing only the administrativeInformation section.
      2. Convert OrderedDicts to dicts.
      3. Wrap the data in a top-level AdministrativeInformation structure.
      4. Dump the resulting instance to a YAML file.
      5. Validate the instance using a schema validator.
      6. Load the YAML instance with YAMLLoader.

    Parameters:
        input_path (str): Path to the input JSON file.
        schema_path (str): Path to the YAML schema file.
        output_yaml_path (str): Path where the output YAML file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during validation or instance loading.
    """
    # Load the minimal JSON instance (containing only administrativeInformation)
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    data = odict_to_dict(data)

    # Build the nested instance structure.
    instance_data = build_minimal_epd_structure(data)

    # Dump to YAML
    with open(output_yaml_path, "w", encoding="utf-8") as out:
        yaml.dump(instance_data, out, sort_keys=False, allow_unicode=True)
    print(f"administrativeInformation EPD YAML instance written to: {output_yaml_path}")

    # Validate using the Validator
    try:
        validator = Validator(schema_path, strict=False)
        report = validator.validate(instance_data, "AdministrativeInformation")
        if report.results:
            print("Validation errors:")
            for result in report.results:
                print(result.message)
        else:
            print("The AdministrativeInformation instance is valid!")
    except Exception as e:
        print("Exception during validation:")
        print(e)
        raise

    # Load the YAML instance using YAMLLoader
    try:
        loader = YAMLLoader()
        # If the instance is wrapped in a "administrativeInformation" key, unwrap it.
        if "administrativeInformation" in instance_data:
            instance_data = instance_data["administrativeInformation"]
        loader.load(instance_data, target_class=AdministrativeInformation)
        print("Instance loaded successfully!")
    except Exception as e:
        print("Exception during instance loading via YAMLLoader:")
        print(e)
        raise


# Usage
input_file = "../linkml/data/json/instance_administrativeInformation.json"
schema_file = "../linkml/data/yaml/linkml_administrativeInformation_schema.yaml"
output_yaml = "../linkml/data/yaml/linkml_administrativeInformation_instance.yaml"

process_instance(input_file, schema_file, output_yaml)

administrativeInformation EPD YAML instance written to: ../linkml/data/yaml/linkml_administrativeInformation_instance.yaml
The AdministrativeInformation instance is valid!
Instance loaded successfully!


In [30]:
# Loading and RDF (JSON-LD, Turtle) Generation administrativeInformation EPD YAML Instance

import yaml
from linkml_runtime.dumpers import RDFLibDumper
from linkml_runtime.loaders import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView
from data.py.linkml_administrativeInformation_schema import AdministrativeInformation


def generate_rdf(
    schema_path: str,
    instance_path: str,
    output_jsonld_path: str,
    output_turtle_path: str,
) -> None:
    """
    Load a validated LinkML YAML instance, create a SchemaView, and dump the instance to RDF in JSON-LD and Turtle formats.

    Steps:
        1. Load the instance from a YAML file.
        2. Create a SchemaView from the schema file.
        3. Generate RDF in JSON-LD and Turtle formats.
        4. Write the RDF to files.

    Parameters:
        schema_path (str): Path to the LinkML YAML schema file.
        instance_path (str): Path to the validated LinkML YAML instance file.
        output_jsonld_path (str): Path where the output JSON-LD file will be written.
        output_turtle_path (str): Path where the output Turtle file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during instance loading, SchemaView creation, or RDF generation.
    """
    # Load instance from file
    try:
        with open(instance_path, "r", encoding="utf-8") as f:
            loaded_data = yaml.safe_load(f)  # This is now a Python dict
        # Extract "administrativeInformation"
        if "administrativeInformation" in loaded_data:
            loaded_data = loaded_data["administrativeInformation"]
        # Feed loaded_data dictionary into the loader
        instance_obj = YAMLLoader().load(
            loaded_data, target_class=AdministrativeInformation
        )
        print(f"Instance loaded from: {instance_path}")
    except Exception as e:
        print("Exception during instance loading:")
        print(e)
        raise

    # Create SchemaView
    try:
        sv = SchemaView(schema_path)
        print("SchemaView created successfully.")
    except Exception as e:
        print("Exception during SchemaView creation:")
        print(e)
        raise

    # Generate and write RDF (JSON-LD, Turtle)
    try:
        dumper = RDFLibDumper()
        # JSON‑LD format
        # rdf_jsonld = dumper.dumps(instance_obj, schemaview=sv, fmt="json-ld")
        # with open(output_jsonld_path, "w", encoding="utf-8") as rdf_file:
        #     rdf_file.write(rdf_jsonld)
        # print(f"RDF (JSON‑LD) successfully written to: {output_jsonld_path}")
        # Turtle format
        rdf_turtle = dumper.dumps(instance_obj, schemaview=sv, fmt="turtle")
        with open(output_turtle_path, "w", encoding="utf-8") as ttl_file:
            ttl_file.write(rdf_turtle)
        print(f"RDF (Turtle) successfully written to: {output_turtle_path}")
    except Exception as e:
        print("Exception during RDF generation:")
        print(e)
        raise


# Example usage:
schema_path = "../linkml/data/yaml/linkml_administrativeInformation_schema.yaml"
instance_path = "../linkml/data/yaml/linkml_administrativeInformation_instance.yaml"
output_jsonld_path = (
    "../linkml/data/rdf/linkml_administrativeInformation_instance.jsonld"
)
output_turtle_path = "../linkml/data/rdf/linkml_administrativeInformation_instance.ttl"

generate_rdf(schema_path, instance_path, output_jsonld_path, output_turtle_path)

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Instance loaded from: ../linkml/data/yaml/linkml_administrativeInformation_instance.yaml
SchemaView created successfully.
RDF (Turtle) successfully written to: ../linkml/data/rdf/linkml_administrativeInformation_instance.ttl


# Generate only the `exchanges` part of the Schema and JSON Instance


In [21]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

sd_yaml_schema = "../linkml/data/yaml/linkml_shared_definitions.yaml"
sd_python_file = "../linkml/data/py/linkml_shared_definitions.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=sd_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()


with open(sd_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {sd_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_shared_definitions.py.


In [22]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

ex_yaml_schema = "../linkml/data/yaml/linkml_exchanges_schema.yaml"
ex_python_file = "../linkml/data/py/linkml_exchanges_schema.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=ex_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()

with open(ex_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {ex_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_exchanges_schema.py.


In [24]:
# Instance Generation and Loading exchanges EPD YAML

import json
import yaml
from collections import OrderedDict
from data.py.linkml_exchanges_schema import Exchanges
from linkml.validator import Validator
from linkml_runtime.loaders.yaml_loader import YAMLLoader


def odict_to_dict(obj):
    """
    Recursively convert an OrderedDict (or list/dict containing OrderedDicts) to a standard dict.

    Parameters:
        obj: An OrderedDict, list, or dict potentially containing OrderedDicts.

    Returns:
        A new structure where all OrderedDict instances are replaced with standard dicts.
    """
    if isinstance(obj, OrderedDict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [odict_to_dict(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    else:
        return obj


def build_minimal_epd_structure(data):
    """
    Build a minimal nested dictionary according to our minimal schema.
    This function assumes that the JSON instance already contains a "exchanges" key.

    Parameters:
        data (dict): A dictionary loaded from a JSON file.

    Returns:
        dict: A dictionary structured with a top-level 'exchanges' key.
    """
    instance_dict = {"exchanges": data.get("exchanges", {})}
    return instance_dict


def process_instance(input_path, schema_path, output_yaml_path):
    """
    Process an instance by loading a minimal JSON instance, converting it, dumping it to YAML,
    validating it against a schema, and loading it with YAMLLoader.

    Steps:
      1. Load a minimal JSON instance containing only the exchanges section.
      2. Convert OrderedDicts to dicts.
      3. Wrap the data in a top-level Exchanges structure.
      4. Dump the resulting instance to a YAML file.
      5. Validate the instance using a schema validator.
      6. Load the YAML instance with YAMLLoader.

    Parameters:
        input_path (str): Path to the input JSON file.
        schema_path (str): Path to the YAML schema file.
        output_yaml_path (str): Path where the output YAML file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during validation or instance loading.
    """
    # Load the minimal JSON instance (containing only exchanges)
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    data = odict_to_dict(data)

    # Build the nested instance structure.
    instance_data = build_minimal_epd_structure(data)

    # Dump to YAML
    with open(output_yaml_path, "w", encoding="utf-8") as out:
        yaml.dump(instance_data, out, sort_keys=False, allow_unicode=True)
    print(f"exchanges EPD YAML instance written to: {output_yaml_path}")

    # Validate using the Validator
    try:
        validator = Validator(schema_path, strict=False)
        report = validator.validate(instance_data, "Exchanges")
        if report.results:
            print("Validation errors:")
            for result in report.results:
                print(result.message)
        else:
            print("The Exchanges instance is valid!")
    except Exception as e:
        print("Exception during validation:")
        print(e)
        raise

    # Load the YAML instance using YAMLLoader
    try:
        loader = YAMLLoader()
        # If the instance is wrapped in a "exchanges" key, unwrap it.
        if "exchanges" in instance_data:
            instance_data = instance_data["exchanges"]
        loader.load(instance_data, target_class=Exchanges)
        print("Instance loaded successfully!")
    except Exception as e:
        print("Exception during instance loading via YAMLLoader:")
        print(e)
        raise


# Usage
input_file = "../linkml/data/json/instance_exchanges.json"
schema_file = "../linkml/data/yaml/linkml_exchanges_schema.yaml"
output_yaml = "../linkml/data/yaml/linkml_exchanges_instance.yaml"

process_instance(input_file, schema_file, output_yaml)

exchanges EPD YAML instance written to: ../linkml/data/yaml/linkml_exchanges_instance.yaml
The Exchanges instance is valid!
Instance loaded successfully!


In [25]:
# Loading and RDF (JSON-LD, Turtle) Generation exchanges EPD YAML Instance

import yaml
from linkml_runtime.dumpers import RDFLibDumper
from linkml_runtime.loaders import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView
from data.py.linkml_exchanges_schema import Exchanges


def generate_rdf(
    schema_path: str,
    instance_path: str,
    output_jsonld_path: str,
    output_turtle_path: str,
) -> None:
    """
    Load a validated LinkML YAML instance, create a SchemaView, and dump the instance to RDF in JSON-LD and Turtle formats.

    Steps:
        1. Load the instance from a YAML file.
        2. Create a SchemaView from the schema file.
        3. Generate RDF in JSON-LD and Turtle formats.
        4. Write the RDF to files.

    Parameters:
        schema_path (str): Path to the LinkML YAML schema file.
        instance_path (str): Path to the validated LinkML YAML instance file.
        output_jsonld_path (str): Path where the output JSON-LD file will be written.
        output_turtle_path (str): Path where the output Turtle file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during instance loading, SchemaView creation, or RDF generation.
    """
    # Load instance from file
    try:
        with open(instance_path, "r", encoding="utf-8") as f:
            loaded_data = yaml.safe_load(f)  # This is now a Python dict
        # Extract "exchanges"
        if "exchanges" in loaded_data:
            loaded_data = loaded_data["exchanges"]
        # Feed loaded_data dictionary into the loader
        instance_obj = YAMLLoader().load(loaded_data, target_class=Exchanges)
        print(f"Instance loaded from: {instance_path}")
    except Exception as e:
        print("Exception during instance loading:")
        print(e)
        raise

    # Create SchemaView
    try:
        sv = SchemaView(schema_path)
        print("SchemaView created successfully.")
    except Exception as e:
        print("Exception during SchemaView creation:")
        print(e)
        raise

    # Generate and write RDF (JSON-LD, Turtle)
    try:
        dumper = RDFLibDumper()
        # JSON‑LD format
        # rdf_jsonld = dumper.dumps(instance_obj, schemaview=sv, fmt="json-ld")
        # with open(output_jsonld_path, "w", encoding="utf-8") as rdf_file:
        #     rdf_file.write(rdf_jsonld)
        # print(f"RDF (JSON‑LD) successfully written to: {output_jsonld_path}")
        # Turtle format
        rdf_turtle = dumper.dumps(instance_obj, schemaview=sv, fmt="turtle")
        with open(output_turtle_path, "w", encoding="utf-8") as ttl_file:
            ttl_file.write(rdf_turtle)
        print(f"RDF (Turtle) successfully written to: {output_turtle_path}")
    except Exception as e:
        print("Exception during RDF generation:")
        print(e)
        raise


# Example usage:
schema_path = "../linkml/data/yaml/linkml_exchanges_schema.yaml"
instance_path = "../linkml/data/yaml/linkml_exchanges_instance.yaml"
output_jsonld_path = "../linkml/data/rdf/linkml_exchanges_instance.jsonld"
output_turtle_path = "../linkml/data/rdf/linkml_exchanges_instance.ttl"

generate_rdf(schema_path, instance_path, output_jsonld_path, output_turtle_path)

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Instance loaded from: ../linkml/data/yaml/linkml_exchanges_instance.yaml
SchemaView created successfully.
RDF (Turtle) successfully written to: ../linkml/data/rdf/linkml_exchanges_instance.ttl


# Generate only the `LCIAResults` part of the Schema and JSON Instance


In [9]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

sd_yaml_schema = "../linkml/data/yaml/linkml_shared_definitions.yaml"
sd_python_file = "../linkml/data/py/linkml_shared_definitions.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=sd_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()


with open(sd_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {sd_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_shared_definitions.py.


In [11]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

ai_yaml_schema = "../linkml/data/yaml/linkml_lciaResults_schema.yaml"
ai_python_file = "../linkml/data/py/linkml_lciaResults_schema.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=ai_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()

with open(ai_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {ai_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_lciaResults_schema.py.


In [13]:
# Instance Generation and Loading LCIAResults EPD YAML

import json
import yaml
from collections import OrderedDict
from data.py.linkml_lciaResults_schema import LCIAResults
from linkml.validator import Validator
from linkml_runtime.loaders.yaml_loader import YAMLLoader


def odict_to_dict(obj):
    """
    Recursively convert an OrderedDict (or list/dict containing OrderedDicts) to a standard dict.

    Parameters:
        obj: An OrderedDict, list, or dict potentially containing OrderedDicts.

    Returns:
        A new structure where all OrderedDict instances are replaced with standard dicts.
    """
    if isinstance(obj, OrderedDict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [odict_to_dict(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    else:
        return obj


def build_minimal_epd_structure(data):
    """
    Build a minimal nested dictionary according to our minimal schema.
    This function assumes that the JSON instance already contains a "LCIAResults" key.

    Parameters:
        data (dict): A dictionary loaded from a JSON file.

    Returns:
        dict: A dictionary structured with a top-level 'LCIAResults' key.
    """
    instance_dict = {"LCIAResults": data.get("LCIAResults", {})}
    return instance_dict


def process_instance(input_path, schema_path, output_yaml_path):
    """
    Process an instance by loading a minimal JSON instance, converting it, dumping it to YAML,
    validating it against a schema, and loading it with YAMLLoader.

    Steps:
      1. Load a minimal JSON instance containing only the LCIAResults section.
      2. Convert OrderedDicts to dicts.
      3. Wrap the data in a top-level LCIAResults structure.
      4. Dump the resulting instance to a YAML file.
      5. Validate the instance using a schema validator.
      6. Load the YAML instance with YAMLLoader.

    Parameters:
        input_path (str): Path to the input JSON file.
        schema_path (str): Path to the YAML schema file.
        output_yaml_path (str): Path where the output YAML file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during validation or instance loading.
    """
    # Load the minimal JSON instance (containing only LCIAResults)
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    data = odict_to_dict(data)

    # Build the nested instance structure.
    instance_data = build_minimal_epd_structure(data)

    # Dump to YAML
    with open(output_yaml_path, "w", encoding="utf-8") as out:
        yaml.dump(instance_data, out, sort_keys=False, allow_unicode=True)
    print(f"LCIAResults EPD YAML instance written to: {output_yaml_path}")

    # Validate using the Validator
    try:
        validator = Validator(schema_path, strict=False)
        report = validator.validate(instance_data, "LCIAResults")
        if report.results:
            print("Validation errors:")
            for result in report.results:
                print(result.message)
        else:
            print("The LCIAResults instance is valid!")
    except Exception as e:
        print("Exception during validation:")
        print(e)
        raise

    # Load the YAML instance using YAMLLoader
    try:
        loader = YAMLLoader()
        # If the instance is wrapped in a "LCIAResults" key, unwrap it.
        if "LCIAResults" in instance_data:
            instance_data = instance_data["LCIAResults"]
        loader.load(instance_data, target_class=LCIAResults)
        print("Instance loaded successfully!")
    except Exception as e:
        print("Exception during instance loading via YAMLLoader:")
        print(e)
        raise


# Usage
input_file = "../linkml/data/json/instance_LCIAResults.json"
schema_file = "../linkml/data/yaml/linkml_LCIAResults_schema.yaml"
output_yaml = "../linkml/data/yaml/linkml_LCIAResults_instance.yaml"

process_instance(input_file, schema_file, output_yaml)

LCIAResults EPD YAML instance written to: ../linkml/data/yaml/linkml_LCIAResults_instance.yaml
The LCIAResults instance is valid!
Instance loaded successfully!


In [14]:
# Loading and RDF (JSON-LD, Turtle) Generation LCIAResults EPD YAML Instance

import yaml
from linkml_runtime.dumpers import RDFLibDumper
from linkml_runtime.loaders import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView
from data.py.linkml_lciaResults_schema import LCIAResults


def generate_rdf(
    schema_path: str,
    instance_path: str,
    output_jsonld_path: str,
    output_turtle_path: str,
) -> None:
    """
    Load a validated LinkML YAML instance, create a SchemaView, and dump the instance to RDF in JSON-LD and Turtle formats.

    Steps:
        1. Load the instance from a YAML file.
        2. Create a SchemaView from the schema file.
        3. Generate RDF in JSON-LD and Turtle formats.
        4. Write the RDF to files.

    Parameters:
        schema_path (str): Path to the LinkML YAML schema file.
        instance_path (str): Path to the validated LinkML YAML instance file.
        output_jsonld_path (str): Path where the output JSON-LD file will be written.
        output_turtle_path (str): Path where the output Turtle file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during instance loading, SchemaView creation, or RDF generation.
    """
    # Load instance from file
    try:
        with open(instance_path, "r", encoding="utf-8") as f:
            loaded_data = yaml.safe_load(f)  # This is now a Python dict
        # Extract "LCIAResults"
        if "LCIAResults" in loaded_data:
            loaded_data = loaded_data["LCIAResults"]
        # Feed loaded_data dictionary into the loader
        instance_obj = YAMLLoader().load(loaded_data, target_class=LCIAResults)
        print(f"Instance loaded from: {instance_path}")
    except Exception as e:
        print("Exception during instance loading:")
        print(e)
        raise

    # Create SchemaView
    try:
        sv = SchemaView(schema_path)
        print("SchemaView created successfully.")
    except Exception as e:
        print("Exception during SchemaView creation:")
        print(e)
        raise

    # Generate and write RDF (JSON-LD, Turtle)
    try:
        dumper = RDFLibDumper()
        # # JSON‑LD format
        # rdf_jsonld = dumper.dumps(instance_obj, schemaview=sv, fmt="json-ld")
        # with open(output_jsonld_path, "w", encoding="utf-8") as rdf_file:
        #     rdf_file.write(rdf_jsonld)
        # print(f"RDF (JSON‑LD) successfully written to: {output_jsonld_path}")
        # Turtle format
        rdf_turtle = dumper.dumps(instance_obj, schemaview=sv, fmt="turtle")
        with open(output_turtle_path, "w", encoding="utf-8") as ttl_file:
            ttl_file.write(rdf_turtle)
        print(f"RDF (Turtle) successfully written to: {output_turtle_path}")
    except Exception as e:
        print("Exception during RDF generation:")
        print(e)
        raise

# Example usage:
schema_path = "../linkml/data/yaml/linkml_LCIAResults_schema.yaml"
instance_path = "../linkml/data/yaml/linkml_LCIAResults_instance.yaml"
output_jsonld_path = "../linkml/data/rdf/linkml_LCIAResults_instance.jsonld"
output_turtle_path = "../linkml/data/rdf/linkml_LCIAResults_instance.ttl"

generate_rdf(schema_path, instance_path, output_jsonld_path, output_turtle_path)

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Instance loaded from: ../linkml/data/yaml/linkml_LCIAResults_instance.yaml
SchemaView created successfully.
RDF (Turtle) successfully written to: ../linkml/data/rdf/linkml_LCIAResults_instance.ttl


# Generate only the `otherAttributes` part of the Schema and JSON Instance

In [1]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

ai_yaml_schema = "../linkml/data/yaml/linkml_otherAttributes_schema.yaml"
ai_python_file = "../linkml/data/py/linkml_otherAttributes_schema.py"

generator = PythonGenerator(schema=ai_yaml_schema,)

python_code = generator.serialize()

with open(ai_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {ai_python_file}.")

KeyError: 'string'

In [1]:
# Instance Generation and Loading otherAttributes EPD YAML

import json
import yaml
from collections import OrderedDict
from data.py.linkml_otherAttributes_schema import OtherAttributes
from linkml.validator import Validator
from linkml_runtime.loaders.yaml_loader import YAMLLoader


def odict_to_dict(obj):
    """
    Recursively convert an OrderedDict (or list/dict containing OrderedDicts) to a standard dict.

    Parameters:
        obj: An OrderedDict, list, or dict potentially containing OrderedDicts.

    Returns:
        A new structure where all OrderedDict instances are replaced with standard dicts.
    """
    if isinstance(obj, OrderedDict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [odict_to_dict(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    else:
        return obj


def build_minimal_epd_structure(data):
    """
    Build a minimal nested dictionary according to our minimal schema.
    This function assumes that the JSON instance already contains a "otherAttributes" key.

    Parameters:
        data (dict): A dictionary loaded from a JSON file.

    Returns:
        dict: A dictionary structured with a top-level 'otherAttributes' key.
    """
    instance_dict = {"otherAttributes": data.get("otherAttributes", {})}
    return instance_dict


def process_instance(input_path, schema_path, output_yaml_path):
    """
    Process an instance by loading a minimal JSON instance, converting it, dumping it to YAML,
    validating it against a schema, and loading it with YAMLLoader.

    Steps:
      1. Load a minimal JSON instance containing only the otherAttributes section.
      2. Convert OrderedDicts to dicts.
      3. Wrap the data in a top-level OtherAttributes structure.
      4. Dump the resulting instance to a YAML file.
      5. Validate the instance using a schema validator.
      6. Load the YAML instance with YAMLLoader.

    Parameters:
        input_path (str): Path to the input JSON file.
        schema_path (str): Path to the YAML schema file.
        output_yaml_path (str): Path where the output YAML file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during validation or instance loading.
    """
    # Load the minimal JSON instance (containing only otherAttributes)
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    data = odict_to_dict(data)

    # Build the nested instance structure.
    instance_data = build_minimal_epd_structure(data)

    # Dump to YAML
    with open(output_yaml_path, "w", encoding="utf-8") as out:
        yaml.dump(instance_data, out, sort_keys=False, allow_unicode=True)
    print(f"otherAttributes EPD YAML instance written to: {output_yaml_path}")

    # Validate using the Validator
    try:
        validator = Validator(schema_path, strict=False)
        report = validator.validate(instance_data, "OtherAttributes")
        if report.results:
            print("Validation errors:")
            for result in report.results:
                print(result.message)
        else:
            print("The OtherAttributes instance is valid!")
    except Exception as e:
        print("Exception during validation:")
        print(e)
        raise

    # Load the YAML instance using YAMLLoader
    try:
        loader = YAMLLoader()
        # If the instance is wrapped in a "otherAttributes" key, unwrap it.
        if "otherAttributes" in instance_data:
            instance_data = instance_data["otherAttributes"]
        loader.load(instance_data, target_class=OtherAttributes)
        print("Instance loaded successfully!")
    except Exception as e:
        print("Exception during instance loading via YAMLLoader:")
        print(e)
        raise


# Usage
input_file = "../linkml/data/json/instance_otherAttributes.json"
schema_file = "../linkml/data/yaml/linkml_otherAttributes_schema.yaml"
output_yaml = "../linkml/data/yaml/linkml_otherAttributes_instance.yaml"

process_instance(input_file, schema_file, output_yaml)


otherAttributes EPD YAML instance written to: ../linkml/data/yaml/linkml_otherAttributes_instance.yaml
Exception during validation:
 Unknown argument: key_name = True


ValueError:  Unknown argument: key_name = True

In [ ]:
# Loading and RDF (JSON-LD, Turtle) Generation LCIAResults EPD YAML Instance

import yaml
from linkml_runtime.dumpers import RDFLibDumper
from linkml_runtime.loaders import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView
from data.py.linkml_LCIAResults_schema import LCIAResults


def generate_rdf(
    schema_path: str,
    instance_path: str,
    output_jsonld_path: str,
    output_turtle_path: str,
) -> None:
    """
    Load a validated LinkML YAML instance, create a SchemaView, and dump the instance to RDF in JSON-LD and Turtle formats.

    Steps:
        1. Load the instance from a YAML file.
        2. Create a SchemaView from the schema file.
        3. Generate RDF in JSON-LD and Turtle formats.
        4. Write the RDF to files.

    Parameters:
        schema_path (str): Path to the LinkML YAML schema file.
        instance_path (str): Path to the validated LinkML YAML instance file.
        output_jsonld_path (str): Path where the output JSON-LD file will be written.
        output_turtle_path (str): Path where the output Turtle file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during instance loading, SchemaView creation, or RDF generation.
    """
    # Load instance from file
    try:
        with open(instance_path, "r", encoding="utf-8") as f:
            loaded_data = yaml.safe_load(f)  # This is now a Python dict
        # Extract "LCIAResults"
        if "LCIAResults" in loaded_data:
            loaded_data = loaded_data["LCIAResults"]
        # Feed loaded_data dictionary into the loader
        instance_obj = YAMLLoader().load(loaded_data, target_class=LCIAResults)
        print(f"Instance loaded from: {instance_path}")
    except Exception as e:
        print("Exception during instance loading:")
        print(e)
        raise

    # Create SchemaView
    try:
        sv = SchemaView(schema_path)
        print("SchemaView created successfully.")
    except Exception as e:
        print("Exception during SchemaView creation:")
        print(e)
        raise

    # Generate and write RDF (JSON-LD, Turtle)
    try:
        dumper = RDFLibDumper()
        # # JSON‑LD format
        # rdf_jsonld = dumper.dumps(instance_obj, schemaview=sv, fmt="json-ld")
        # with open(output_jsonld_path, "w", encoding="utf-8") as rdf_file:
        #     rdf_file.write(rdf_jsonld)
        # print(f"RDF (JSON‑LD) successfully written to: {output_jsonld_path}")
        # Turtle format
        rdf_turtle = dumper.dumps(instance_obj, schemaview=sv, fmt="turtle")
        with open(output_turtle_path, "w", encoding="utf-8") as ttl_file:
            ttl_file.write(rdf_turtle)
        print(f"RDF (Turtle) successfully written to: {output_turtle_path}")
    except Exception as e:
        print("Exception during RDF generation:")
        print(e)
        raise

# Example usage:
schema_path = "../linkml/data/yaml/linkml_LCIAResults_schema.yaml"
instance_path = "../linkml/data/yaml/linkml_LCIAResults_instance.yaml"
output_jsonld_path = "../linkml/data/rdf/linkml_LCIAResults_instance.jsonld"
output_turtle_path = "../linkml/data/rdf/linkml_LCIAResults_instance.ttl"

generate_rdf(schema_path, instance_path, output_jsonld_path, output_turtle_path)

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Instance loaded from: ../linkml/data/yaml/linkml_LCIAResults_instance.yaml
SchemaView created successfully.
RDF (Turtle) successfully written to: ../linkml/data/rdf/linkml_LCIAResults_instance.ttl


# Generate the whole schema and JSON Instance


In [5]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

sd_yaml_schema = "../linkml/data/yaml/linkml_shared_definitions.yaml"
sd_python_file = "../linkml/data/py/linkml_shared_definitions.py"

# Define an import map so that types like "String", "Boolean", etc.
# are resolved to their LinkML runtime equivalents.
import_map = {
    "Boolean": "linkml_runtime.linkml_model.types.Boolean",
    "String": "linkml_runtime.linkml_model.types.String",
    "Integer": "linkml_runtime.linkml_model.types.Integer",
    "Float": "linkml_runtime.linkml_model.types.Float",
    "dateTime": "linkml_runtime.linkml_model.types.dateTime",
    # "Version": "linkml_runtime.linkml_model.types.Version", # Not in LinkML runtime
    # "anyURI": "linkml_runtime.linkml_model.types.anyURI"  # Not in LinkML runtime
}

generator = PythonGenerator(
    schema=sd_yaml_schema,
    gen_slots=True,
    gen_classvars=True,
    mergeimports=True,
    metadata=True,
    genmeta=False,
    importmap=import_map,
)


python_code = generator.serialize()


with open(sd_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {sd_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_shared_definitions.py.


In [6]:
# Produce Python module from YAML schema using LinkML's Python generator

from linkml.generators.pythongen import PythonGenerator

ai_yaml_schema = "../linkml/data/yaml/linkml_processDataSet_schema.yaml"
ai_python_file = "../linkml/data/py/linkml_processDataSet_schema.py"

generator = PythonGenerator(schema=ai_yaml_schema)

python_code = generator.serialize()

with open(ai_python_file, "w", encoding="utf-8") as f:
    f.write(python_code)

print(f"Python model generated and written to {ai_python_file}.")

c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Python model generated and written to ../linkml/data/py/linkml_processDataSet_schema.py.


In [7]:
import json
import yaml
from collections import OrderedDict
from data.py.linkml_processDataSet_schema import ProcessDataSet
from linkml.validator import Validator
from linkml_runtime.loaders.yaml_loader import YAMLLoader

def odict_to_dict(obj):
    """
    Recursively convert an OrderedDict (or list/dict containing OrderedDicts) to a standard dict.
    """
    if isinstance(obj, OrderedDict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [odict_to_dict(item) for item in obj]
    elif isinstance(obj, dict):
        return {k: odict_to_dict(v) for k, v in obj.items()}
    else:
        return obj

def process_instance(input_path, schema_path, output_yaml_path):
    """
    Process an instance by loading a JSON file that is already structured
    at the top level for the ProcessDataSet class. Example top-level JSON keys:
        {
          "processInformation": { ... },
          "modellingAndValidation": { ... },
          ...
          "version": "1.1"
        }

    Steps:
      1. Load and parse the JSON directly into a dict.
      2. Dump the dict to a YAML file for inspection (optional).
      3. Validate the dict against the ProcessDataSet class in your schema.
      4. Load the dict as a ProcessDataSet object using YAMLLoader.
    """
    # 1. Load the JSON instance
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # (Optional) Convert OrderedDicts to normal dicts if needed
    data = odict_to_dict(data)

    # 2. Dump the data to a YAML file (for debugging or pipeline usage)
    with open(output_yaml_path, "w", encoding="utf-8") as out:
        yaml.dump(data, out, sort_keys=False, allow_unicode=True)
    print(f"ProcessDataSet EPD YAML instance written to: {output_yaml_path}")

    # 3. Validate the dict using the LinkML Validator
    try:
        validator = Validator(schema_path, strict=False)
        report = validator.validate(data, "ProcessDataSet")
        if report.results:
            print("Validation errors:")
            for result in report.results:
                print(result.message)
        else:
            print("The ProcessDataSet instance is valid!")
    except Exception as e:
        print("Exception during validation:")
        print(e)
        raise

    # 4. Load the dict as a ProcessDataSet object
    try:
        loader = YAMLLoader()
        obj = loader.load(data, target_class=ProcessDataSet)
        print("Instance loaded successfully as ProcessDataSet object!")
        # You can now use `obj` in your code, e.g. print(obj)
    except Exception as e:
        print("Exception during instance loading via YAMLLoader:")
        print(e)
        raise


# Usage example
input_file = "../data/pipeline2/json/epds/0db12903-1403-4c9a-817e-b48299d17aba_RN_ID.json"
# input_file = "../linkml/data/json/5b6b44e0-f5e4-451f-54a3-08dcec2f0f89_renamedScript_newID.json"
schema_file = "../linkml/data/yaml/linkml_processDataSet_schema.yaml"
output_yaml = "../linkml/data/yaml/linkml_processDataSet_instance.yaml"

process_instance(input_file, schema_file, output_yaml)


ProcessDataSet EPD YAML instance written to: ../linkml/data/yaml/linkml_processDataSet_instance.yaml
The ProcessDataSet instance is valid!
Instance loaded successfully as ProcessDataSet object!


In [8]:
# Loading and RDF (JSON-LD, Turtle) Generation processDataSet EPD YAML Instance

import yaml
from linkml_runtime.dumpers import RDFLibDumper
from linkml_runtime.loaders import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView
from data.py.linkml_processDataSet_schema import ProcessDataSet


def generate_rdf(
    schema_path: str,
    instance_path: str,
    output_jsonld_path: str,
    output_turtle_path: str,
) -> None:
    """
    Load a validated LinkML YAML instance, create a SchemaView, and dump the instance to RDF in JSON-LD and Turtle formats.

    Steps:
        1. Load the instance from a YAML file.
        2. Create a SchemaView from the schema file.
        3. Generate RDF in JSON-LD and Turtle formats.
        4. Write the RDF to files.

    Parameters:
        schema_path (str): Path to the LinkML YAML schema file.
        instance_path (str): Path to the validated LinkML YAML instance file.
        output_jsonld_path (str): Path where the output JSON-LD file will be written.
        output_turtle_path (str): Path where the output Turtle file will be written.

    Returns:
        None

    Raises:
        Exception: If an error occurs during instance loading, SchemaView creation, or RDF generation.
    """
    # Load instance from file
    try:
        with open(instance_path, "r", encoding="utf-8") as f:
            loaded_data = yaml.safe_load(f)  # This is now a Python dict
        # Extract "processDataSet"
        if "processDataSet" in loaded_data:
            loaded_data = loaded_data["processDataSet"]
        # Feed loaded_data dictionary into the loader
        instance_obj = YAMLLoader().load(loaded_data, target_class=ProcessDataSet)
        print(f"Instance loaded from: {instance_path}")
    except Exception as e:
        print("Exception during instance loading:")
        print(e)
        raise

    # Create SchemaView
    try:
        sv = SchemaView(schema_path)
        print("SchemaView created successfully.")
    except Exception as e:
        print("Exception during SchemaView creation:")
        print(e)
        raise

    # Generate and write RDF (JSON-LD, Turtle)
    try:
        dumper = RDFLibDumper()
        # JSON‑LD format
        # rdf_jsonld = dumper.dumps(instance_obj, schemaview=sv, fmt="json-ld")
        # with open(output_jsonld_path, "w", encoding="utf-8") as rdf_file:
        #     rdf_file.write(rdf_jsonld)
        # print(f"RDF (JSON‑LD) successfully written to: {output_jsonld_path}")
        # Turtle format
        rdf_turtle = dumper.dumps(instance_obj, schemaview=sv, fmt="turtle")
        with open(output_turtle_path, "w", encoding="utf-8") as ttl_file:
            ttl_file.write(rdf_turtle)
        print(f"RDF (Turtle) successfully written to: {output_turtle_path}")
    except Exception as e:
        print("Exception during RDF generation:")
        print(e)
        raise


# Example usage:
schema_path = "../linkml/data/yaml/linkml_processDataSet_schema.yaml"
instance_path = "../linkml/data/yaml/linkml_processDataSet_instance.yaml"
output_jsonld_path = "../linkml/data/rdf/linkml_processDataSet_instance.jsonld"
output_turtle_path = "../linkml/data/rdf/linkml_processDataSet_instance.ttl"

generate_rdf(schema_path, instance_path, output_jsonld_path, output_turtle_path)


c:\Users\Georgi\.conda\envs\linkml-env\Lib\site-packages\linkml_runtime\linkml_model\model\schema\types does not look like a valid URI, trying to serialize this will break.


Instance loaded from: ../linkml/data/yaml/linkml_processDataSet_instance.yaml
SchemaView created successfully.
RDF (Turtle) successfully written to: ../linkml/data/rdf/linkml_processDataSet_instance.ttl
